# Setup Awal

In [ ]:
# @title ♻️ Mount to Drive

from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# @title 🚀 Clone Repo & Setup Lingkungan
!git clone https://github.com/lIlSkaSkaSkalIl/video_toolkit.git /content/video_toolkit > /dev/null
!apt install ffmpeg -y > /dev/null
!pip install tqdm > /dev/null

import os
import sys

# Tambahkan path repo
sys.path.append("/content")

# Path utama media_toolkit
MEDIA_DIR = "/content/media_toolkit"
INPUT_DIR = os.path.join(MEDIA_DIR, "input")
TEMP_DIR = os.path.join(MEDIA_DIR, "temp_chunks")
OUTPUT_DIR = os.path.join(MEDIA_DIR, "output_parts")
METADATA_DIR = os.path.join(MEDIA_DIR, "metadata")

# Buat direktori
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(METADATA_DIR, exist_ok=True)

def log(msg):
    print(f"📌 {msg}")

log("✅ Repo berhasil di-clone dan lingkungan siap digunakan.")

# Split Video

In [ ]:
# @title 📂 Ambil Video dari Google Drive

import shutil

gdrive_file_path = ""  # @param {type:"string"}
original_filename = os.path.basename(gdrive_file_path)
local_video_path = os.path.join(INPUT_DIR, original_filename)

log("⏳ Menyalin File, Mohon tunggu...")
shutil.copy(gdrive_file_path, local_video_path)
log(f"✅ File disalin ke: {local_video_path}")

In [ ]:
# @title 📋 Ambil Metadata & Simpan

from video_toolkit.splitter.metadata_extractor import extract_metadata, save_metadata, summarize_metadata
from video_toolkit.utils.messages import show_metadata_summary

video_path = local_video_path
metadata = extract_metadata(video_path)

json_path = os.path.join(METADATA_DIR, "input_metadata.json")
save_metadata(metadata, json_path)
log(f"📄 Metadata disimpan di: {json_path}")

duration, size_mb, bitrate_kbps = summarize_metadata(metadata)

# ✅ Tampilkan ringkasan metadata dengan fungsi terpusat
show_metadata_summary(duration, size_mb, bitrate_kbps)

In [ ]:
# @title 🧮 Hitung Durasi Maksimal per Part

from video_toolkit.utils.helpers import calculate_max_duration
from video_toolkit.utils.messages import show_split_duration_summary

ukuran_max_gb = 1  # @param {type:"number"}
telegram_mode = False  # @param {type:"boolean"}

input_target_mb = ukuran_max_gb * 1024

max_duration_sec, video_size_mb, TARGET_SIZE_MB, will_split = calculate_max_duration(
    metadata, input_target_mb, telegram_mode
)

if telegram_mode and input_target_mb > 1900:
    log("⚠️ Mode Telegram aktif — membatasi ke 1.9GB")

if not will_split:
    log(f"Ukuran video {video_size_mb:.2f}MB ≤ batas part. Tidak perlu di-split.")
else:
    log(f"Durasi maksimal dihitung berdasarkan bitrate: {int(max_duration_sec)} detik")

# ✅ Tampilkan ringkasan
show_split_duration_summary(video_size_mb, TARGET_SIZE_MB, telegram_mode)

In [ ]:
# @title ✂️📦 Split & Gabungkan Video Menjadi Part

from video_toolkit.splitter.video_splitter import split_video_by_duration
from video_toolkit.splitter.merger import group_chunks_by_size, merge_chunks, get_file_size

# Siapkan nama file dasar dan direktori sementara
base_filename = os.path.splitext(os.path.basename(video_path))[0]

try:
    CHUNK_DURATION = int(max_duration_sec)
except NameError:
    raise ValueError("❌ Anda belum menjalankan Cell Hitung Durasi!")

log(f"✂️ Memulai split video ke potongan {CHUNK_DURATION} detik...")

# Split video menjadi potongan
all_chunks = split_video_by_duration(
    video_path=video_path,
    output_dir=TEMP_DIR,
    base_filename=base_filename,
    total_duration=float(metadata["format"]["duration"]),
    chunk_duration=CHUNK_DURATION
)

log(f"📦 Total chunk dibuat: {len(all_chunks)}")

# Gabungkan potongan menjadi part sesuai ukuran target
groups = group_chunks_by_size(all_chunks, TARGET_SIZE_MB)
log(f"🧩 {len(groups)} part akan dibuat berdasarkan ukuran target.")

for i, group in enumerate(groups, 1):
    output_file = os.path.join(OUTPUT_DIR, f"{base_filename}_part_{i:03d}.mp4")
    log(f"🔗 Menggabungkan {len(group)} chunk ke dalam {os.path.basename(output_file)} ...")
    merge_chunks(group, output_file, temp_dir=TEMP_DIR)
    size = get_file_size(output_file)
    log(f"✅ {os.path.basename(output_file)} selesai ({size:.2f} MB)")

# Utility

In [ ]:
# @title 🧮 Duration Summary: Split & Merge Results

from video_toolkit.splitter.metadata_extractor import extract_metadata
from video_toolkit.utils.messages import print_part_duration_report
from video_toolkit.utils.helpers import format_duration
import math

# ✅ Ambil metadata durasi asli
original_metadata = extract_metadata(local_video_path)
original_duration = float(original_metadata["format"]["duration"])

# ✅ Hitung durasi tiap part
part_durations_raw = []
for i, group in enumerate(groups, 1):
    output_file = os.path.join(OUTPUT_DIR, f"{base_filename}_part_{i:03d}.mp4")
    part_meta = extract_metadata(output_file)
    part_duration = float(part_meta["format"]["duration"])
    part_durations_raw.append(part_duration)

# ✅ Format durasi
formatted_parts = [format_duration(d) for d in part_durations_raw]
total_merged_duration = sum(part_durations_raw)
difference = math.ceil(abs(original_duration - total_merged_duration))

# ✅ Tampilkan ringkasan (print only)
print_part_duration_report(
    part_durations=formatted_parts,
    total_duration=format_duration(total_merged_duration),
    original_duration=format_duration(original_duration),
    difference=format_duration(difference)
)

In [ ]:
# @title ☁️ Upload File/Folder ke Google Drive
import shutil
import os
from pathlib import Path

source_path = ""  # @param {type:"string"}
gdrive_target_path = ""  # @param {type:"string"}

def log(msg):
    print(f"📌 {msg}")

def upload_to_gdrive(src, dst):
    src_path = Path(src)
    dst_path = Path(dst)

    if not src_path.exists():
        log(f"❌ Sumber tidak ditemukan: {src}")
        return

    os.makedirs(dst, exist_ok=True)

    if src_path.is_file():
        log(f"📤 Menyalin file: {src_path.name}")
        shutil.copy2(src, dst)
        log(f"✅ File berhasil disalin ke: {dst}")
    elif src_path.is_dir():
        log(f"📁 Menyalin isi folder: {src_path}")
        for item in sorted(src_path.glob("*")):
            if item.is_file():
                shutil.copy2(item, dst_path / item.name)
                log(f"✅ {item.name} disalin ke {dst_path}")
        log("📦 Semua file selesai disalin.")
    else:
        log("❌ Path tidak valid.")

upload_to_gdrive(source_path, gdrive_target_path)

In [ ]:
# @title 🎞️ Tampilkan Metadata Lengkap
import os
import json
import subprocess
from pathlib import Path
import glob

input_path = ""  # @param {type:"string"}

def is_video_file(path):
    return Path(path).suffix.lower() in ['.mp4', '.mkv', '.mov', '.avi', '.webm', '.flv', '.ts']

def get_all_video_files(path):
    path_obj = Path(path)
    if path_obj.is_file() and is_video_file(path_obj):
        return [path_obj]
    elif path_obj.is_dir():
        return [p for p in path_obj.glob("*") if is_video_file(p)]
    else:
        return []

def get_metadata_ffprobe(video_path):
    result = subprocess.run([
        "ffprobe", "-v", "error", "-print_format", "json",
        "-show_format", "-show_streams", str(video_path)
    ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    return json.loads(result.stdout)

def print_metadata(metadata, filename):
    print(f"\n🎬 Metadata untuk: {filename}")
    print("────────────────────────────────────────────")

    # Format section
    format_data = metadata.get("format", {})
    print("📦 Format Info:")
    print(f"   ├ Nama File     : {format_data.get('filename', '-')}")
    print(f"   ├ Durasi        : {format_data.get('duration', '-')} detik")
    print(f"   ├ Ukuran        : {int(format_data.get('size', 0)) / (1024 * 1024):.2f} MB")
    print(f"   ├ Bitrate       : {int(format_data.get('bit_rate', 0)) / 1000:.0f} kbps")
    print(f"   └ Format        : {format_data.get('format_name', '-')}")

    # Stream section
    streams = metadata.get("streams", [])
    for i, stream in enumerate(streams):
        codec_type = stream.get("codec_type", "?")
        icon = "🎞️" if codec_type == "video" else "🎵" if codec_type == "audio" else "📄"

        print(f"\n{icon} Stream {i} ({codec_type.upper()}):")
        print(f"   ├ Codec         : {stream.get('codec_name', '-')}")
        if codec_type == "video":
            print(f"   ├ Resolusi      : {stream.get('width', '?')}x{stream.get('height', '?')}")
            print(f"   ├ FPS           : {stream.get('avg_frame_rate', '-')} fps")
        elif codec_type == "audio":
            print(f"   ├ Sample Rate   : {stream.get('sample_rate', '-')} Hz")
            print(f"   ├ Channels      : {stream.get('channels', '-')}")
        print(f"   └ Language      : {stream.get('tags', {}).get('language', 'unknown')}")

video_files = get_all_video_files(input_path)

if not video_files:
    print(f"❌ Tidak ada file video ditemukan di: {input_path}")
else:
    print(f"📂 Ditemukan {len(video_files)} file video:")
    for file in video_files:
        print(f"  ├ {file.name}")

    for video_file in video_files:
        metadata = get_metadata_ffprobe(video_file)
        print_metadata(metadata, video_file.name)